In [9]:
import os
import pandas as pd
import pymysql
import numpy as np

def dev_db_conn():
    """개발 DB 접속 함수"""
    conn = pymysql.connect(
        host = 'db-7ma06.pub-cdb.ntruss.com',
        user = 'redtable',
        password = 'fpemxpdlqmf5491!@#',
        autocommit = True,
        cursorclass= pymysql.cursors.DictCursor)
    return conn


def rt_db_conn():
    """빅데이터센터 DB 접속 함수"""
    conn = pymysql.connect(
        host = '220.95.212.6',
        user = 'red_dev_jiang',
        password = 'redtable211b',
        db = 'redtable_bigdata_center',
        autocommit = True,
        cursorclass= pymysql.cursors.DictCursor)
    conn.query("set character_set_connection=utf8;")
    conn.query("set character_set_server=utf8;")
    conn.query("set character_set_client=utf8;")
    conn.query("set character_set_results=utf8;")
    conn.query("set character_set_database=utf8;")
    return conn


file_path = os.path.join(os.environ["HOMEPATH"], "Downloads/")

# KFOOD전통식당데이터

# RT_KFD_TRDIT_RSTRNT_INFO_202407

## https://www.bigdata-culture.kr/bigdata/user/data_market/detail.do?id=5eabfe90-3b98-11eb-af9a-4b03f0a582d6

In [10]:
with rt_db_conn() as conn:
    sql = """
    SELECT TABLE_NAME, COLUMN_NAME, column_comment
    FROM information_schema.`COLUMNS`
    WHERE TABLE_NAME = 'RT_KFD_TRDIT_RSTRNT_INFO_202407'
    """
    columns = pd.read_sql(sql, conn)
    
# 컬럼명 가져오기
db_cols = ','.join(columns['COLUMN_NAME'])

In [11]:
with dev_db_conn() as conn:
    sql = """
    SELECT T3.SRC_ID AS 'RSTRNT_ID', '소상공인시장진흥공단' AS 'CRTFC_INSTT_NM', 'KOR' AS 'COUNTRY_NM',
      REPLACE(REPLACE(REPLACE(T1.AREA_NM_1, '강원도', '강원특별자치도'), '전라북도', '전북특별자치도'), '제주도', '제주특별자치도') AS 'CTY_NM',
      T1.RSTR_NM AS 'RSTRNT_NM',
      GROUP_CONCAT(DISTINCT T5.LABEL_NM) AS 'RSTRNT_CTGRY_FLAG_CD',
      CONCAT(LEFT(T6.LCNS_NO,8), '000000') AS 'RSTRNT_OPBIZ_DT',
      IF(T1.RSTR_LNNO_ADRES = '', NULL, T1.RSTR_LNNO_ADRES) AS 'RSTRNT_LNM_ADDR',
      IF(T1.RSTR_RDNMADR = '', NULL, T1.RSTR_RDNMADR) AS 'RSTRNT_ROAD_NM_ADDR',
      IF(T1.RSTR_TELNO = '', NULL, REPLACE(T1.RSTR_REPRSNT_TELNO, '-','')) AS 'RSTRNT_TEL_NO',
      T1.RSTR_LA AS 'RSTRNT_LA', T1.RSTR_LO AS 'RSTRNT_LO'
    FROM redtable01.RSTR T1
    INNER JOIN redtable01.RSTR_INDEX T2 ON T1.ID = T2.RSTR_ID
    INNER JOIN redtable01.RSTR_CONN T3 ON T2.RSTR_ID = T3.RSTR_ID
    LEFT JOIN redtable01.RSTR_LABEL T4 ON T3.RSTR_ID = T4.RSTR_ID
    LEFT JOIN redtable01.RSTR_LABEL_MASTER T5 ON T4.LABEL_MASTER_ID = T5.ID
    LEFT JOIN redtable_raw_2022.foodsafety_restaurant T6 ON T1.BSNS_LCNC_NO = T6.LCNS_NO
    INNER JOIN (SELECT RSTR_ID
        FROM redtable_raw_2022.LONGEVITY_SMALL_BUSINESS
        WHERE RSTR_ID IS NOT NULL) T7 ON T3.RSTR_ID = T7.RSTR_ID
    WHERE T2.IS_USE=1 AND T2.IS_OPEN=1 AND T3.SRC='live_db' AND T1.AREA_NM_1 IS NOT NULL AND T1.AREA_NM_1<>''
    AND T1.RSTR_NM IS NOT NULL AND T1.RSTR_NM<>'' AND T1.RSTR_NM NOT LIKE '%？%'
    GROUP BY T3.SRC_ID;
    """
    df = pd.read_sql(sql, conn)

# RSTRNT_OPBIZ_DT 컬럼값 일시_V14 도메인을 사용하는 컬럼의 데이터 길이는 14 여야 한다. (2024-07-19)
# 결측치 및 불필요한 데이터 정리
df = df.fillna(np.nan)
df.replace({np.nan:None, '\\n':None}, inplace=True)

In [12]:
df

,RSTRNT_ID,CRTFC_INSTT_NM,COUNTRY_NM,CTY_NM,RSTRNT_NM,RSTRNT_CTGRY_FLAG_CD,RSTRNT_OPBIZ_DT,RSTRNT_LNM_ADDR,RSTRNT_ROAD_NM_ADDR,RSTRNT_TEL_NO,RSTRNT_LA,RSTRNT_LO
0,10407,소상공인시장진흥공단,KOR,전라남도,소망식당,한식,20160523000000,전라남도 해남군 해남읍 구교리 314-4,전라남도 해남군 해남읍 구교2길 2,None,34.576195,126.594534
1,10509,소상공인시장진흥공단,KOR,전라남도,건봉국밥,한식,19870509000000,전라남도 순천시 인제동 371-1,전라남도 순천시 장평로 65,None,34.945541,127.493955
2,10770,소상공인시장진흥공단,KOR,경상북도,진가네대구갈비,한식,19830538000000,경상북도 경주시 황오동 329-3,경상북도 경주시 북정로 5,0547721384,35.841453,129.215144
3,10792,소상공인시장진흥공단,KOR,경상북도,외바우,한식,20040538000000,경상북도 경주시 안강읍 산대리 2402-6,경상북도 경주시 안강읍 구부랑3길 12,None,36.001121,129.195584
4,10799,소상공인시장진흥공단,KOR,경상북도,토함산식당,한식,19940538000000,경상북도 경주시 하동 582-3,경상북도 경주시 새골길 197,None,35.798611,129.312880
...,...,...,...,...,...,...,...,...,...,...,...,...
761,912280,소상공인시장진흥공단,KOR,울산광역시,교촌치킨,"일식,한식",19990252000000,울산광역시 동구 서부동 113-33,울산광역시 동구 명덕3길 13,0522341080,35.523518,129.431828
762,913563,소상공인시장진흥공단,KOR,인천광역시,형제들감자탕,한식,19960197000000,인천광역시 연수구 동춘동 803-9,인천광역시 연수구 앵고개로103번길 25,0328193075,37.413877,126.658114
763,915167,소상공인시장진흥공단,KOR,인천광역시,하늘가든,한식,19990212000000,인천광역시 옹진군 영흥면 내리 196,인천광역시 옹진군 영흥면 영흥로251번길 24,0328863916,37.257752,126.484816
764,916297,소상공인시장진흥공단,KOR,대구광역시,큰나무집궁중약백숙,한식,20080188000000,대구광역시 달성군 가창면 삼산리 930-1,대구광역시 달성군 가창면 우록길 24,None,35.733641,128.659546


In [13]:
with rt_db_conn() as conn:
    with conn.cursor() as curs:
        sql = f"""
        INSERT INTO redtable_bigdata_center.RT_KFD_TRDIT_RSTRNT_INFO_202407({db_cols})
        VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
        """
        val = df.apply(lambda row:(row["RSTRNT_ID"], row["CRTFC_INSTT_NM"], row["COUNTRY_NM"], row["CTY_NM"],
                                  row["RSTRNT_NM"], row["RSTRNT_CTGRY_FLAG_CD"], row["RSTRNT_OPBIZ_DT"], row["RSTRNT_LNM_ADDR"],
                                  row["RSTRNT_ROAD_NM_ADDR"], row["RSTRNT_TEL_NO"], row["RSTRNT_LA"],row["RSTRNT_LO"]), 
                                  axis=1).tolist()
        curs.executemany(sql, val)